In [1]:
from datetime import datetime
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import pandas as pd
import pickle
from sklearn.metrics import root_mean_squared_error

In [3]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/1', creation_time=1714578727670, experiment_id='1', last_update_time=1714578727670, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1714574213864, experiment_id='0', last_update_time=1714574213864, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
client.create_experiment(name="my-cool-experiment")

client.search_experiments()

[<Experiment: artifact_location='file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/2', creation_time=1716931475488, experiment_id='2', last_update_time=1716931475488, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/1', creation_time=1714578727670, experiment_id='1', last_update_time=1714578727670, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1714574213864, experiment_id='0', last_update_time=1714574213864, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 164b87b2300b4d31997652d119594f87, rmse: 6.3606
run id: 4d13eabad27c4ca0aaa737247afc24d7, rmse: 6.6436


In [6]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = "164b87b2300b4d31997652d119594f87"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716931566102, current_stage='None', description=None, last_updated_timestamp=1716931566102, name='nyc-taxi-regressor', run_id='164b87b2300b4d31997652d119594f87', run_link=None, source='file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/1/164b87b2300b4d31997652d119594f87/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [7]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging
version: 2, stage: None


C:\Users\rebek\AppData\Local\Temp\ipykernel_20272\669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [8]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

C:\Users\rebek\AppData\Local\Temp\ipykernel_20272\4090915774.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.12.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716931566102, current_stage='Staging', description=None, last_updated_timestamp=1716931594110, name='nyc-taxi-regressor', run_id='164b87b2300b4d31997652d119594f87', run_link=None, source='file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/1/164b87b2300b4d31997652d119594f87/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [9]:
date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1716931566102, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2024-05-28', last_updated_timestamp=1716931607036, name='nyc-taxi-regressor', run_id='164b87b2300b4d31997652d119594f87', run_link=None, source='file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/1/164b87b2300b4d31997652d119594f87/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [12]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [13]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [14]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

c:\Users\rebek\.virtualenvs\machine-learning-ops-zoomcamp-yuDfY6b0\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MlflowException: The following failures occurred while downloading one or more artifacts from file:///d:/rebeka/developer/courses/machine-learning-ops-zoomcamp/code/2_mlflow/mlruns/1/164b87b2300b4d31997652d119594f87/artifacts:
##### File preprocessor #####
[WinError 3] The system cannot find the path specified

In [ ]:
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [ ]:
X_test = preprocess(df, dv)

target = "duration"
y_test = df[target].values

In [ ]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

In [ ]:

%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)